In [3]:
import warnings
import copy

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from Functions import pmfg
import seaborn as sns
import networkx as nx


warnings.filterwarnings('ignore')
%matplotlib inline


def rwa(x, weights):
    return np.sum(x * weights)

def P(x):
    return np.minimum(0.1 + 0.9*x, 1)

def Q(D_s):
    return 1. - 2**(-D_s/100.)

def extract_country(x):
    return x[0:2]

def dist_fct(x):
    return np.sqrt(2*(1-np.minimum(x, 1)))

def make_label_dict(labels):
    l = {}
    for i, label in enumerate(labels):
        l[i] = label
    return l

pd.set_option('display.max_columns', 22)

In [190]:
def overlap(x1, x2):
    return np.dot(x1, x2) / np.sqrt(np.dot(x1,x1) * np.dot(x2,x2))

class Bank(object):
    def __init__(self, name, country, holdings, capital):
        self._name = name
        self._country = country
        self._holdings = holdings
        self._capital = capital
        
    def getName(self):
        return self._name
    
    def setName(self, n):
        self._name = n
    
    def getCountry(self):
        return self._country
    
    def setCountry(self, c):
        self._country = c
    
    def getHoldings(self):
        return self._holdings
    
    def setHoldings(self, h):
        self._holdings = h
    
    def getCapital(self):
        return self._capital
    
    def setCapital(self, c):
        self._capital = c
        
    name = property(getName, setName)
    country = property(getCountry, setCountry)
    holdings = property(getHoldings, setHoldings)
    capital = property(getCapital, setCapital)
    
    
    def getOverlap(self, other):
        return overlap(self.getHoldings(), other.getHoldings())
    
    def __str__(self):
        return '{}, from {}'.format(self.name, self.country)
        
class Asset(object):
    def __init__(self, asset_class, name, value, holders, riskweight=None):
        self._asset_class = asset_class
        self._name = name
        self._value = value
        self._holders = holders
        self._riskweight = riskweight
        
    def getName(self):
        return self._name
    
    def getAssetClass(self):
        return self._asset_class
    
    def getValue(self):
        return self._value
    
    def getHolders(self):
        return self._holders
    
    def getRiskWeight(self):
        return self._riskweight
    
    def setRiskWeight(self, r):
        self._riskweight = r
        
    riskweight = property(getRiskWeight, setRiskWeight)
    
    
    def __str__(self):
        return '{}, class {}, total value of {}, risk-weight {}'.format(self._name, self._asset_class, np.round(self._value, 0), self.riskweight)
        
        
class SovereignDebt(Asset):
    def __init__(self, name, value, holders, riskweight=None, shocked=False):
        Asset.__init__(self, "SD", name, value, holders, riskweight=None)
        self._shocked = shocked
    
    def getCountry(self):
        return self._country
    
    def isShocked(self):
        return self._shocked
    
    def shock(self, s):
        self._shocked = s


df = pd.read_excel("Credit risk exposures and exposures to sovereigns a.xlsx", header=1, index_col=0)
df = df.iloc[:, 0:8]
df = df.drop("4. Total retail", axis = 1)
df.columns = ["1_Sov", "2_Fin", "3_Corp", "4-1_Ret_Res", "4-2_Ret_Rev", "4-3_Ret_SME", "5_CRE"]

sd_df = pd.read_excel("EU_SOV_Debt_Bank_Net_2011.xlsx", header=0, index_col=0)
sd_df.index = sd_df.index.rename("SD_Name")
sd_names = [sd.encode('ascii', 'ignore') for sd in sd_df.index]
sd_names_prefix = ["1_Sov_" + sd for sd in sd_names]
sd_df = sd_df.T

cap_df = pd.read_excel("Tier-1 CRs 2010.xlsx", header=2, index_col=1).iloc[:, 2].to_frame("Capital")

df = pd.merge(sd_df, df, left_index=True, right_index=True).rename(columns={"1_Sov":"1_Sov__Other"})
df = pd.merge(df, cap_df, left_index=True, right_index=True)
df.loc[:, "1_Sov__Other"] = df.loc[:, "1_Sov__Other"] - df.loc[:, sd_names].apply(np.sum, axis=1)
df.loc[:, "1_Sov__Other"] = np.maximum(df.loc[:, "1_Sov__Other"], 0.)

banks = {}
for bank in df.index:
    banks[bank] = Bank(bank, bank[0:2], df.ix[bank, :-1], df.ix[bank, -1])
    
assets = {}
for asset in df.iloc[:, :-1].columns:
    if asset in sd_names:
        assets[asset] = SovereignDebt(asset, np.sum(df.loc[:, asset]), df.loc[:, asset]/np.sum(df.loc[:, asset]), None, False)
    else:
        asset_class = asset[asset.find("_")+1:]
        assets[asset] = Asset(asset_class, asset, np.sum(df.loc[:, asset]), df.loc[:, asset]/np.sum(df.loc[:, asset]), None)

In [189]:
#assets["2_Fin"].setRiskWeight(0.02)
print assets["2_Fin"]
assets["2_Fin"].getRiskWeight()

2_Fin, class Fin, total value of 3122243.0, risk-weight None


In [10]:
all_assets_df = df.copy()
all_assets_df.loc[:, "1_Sov"] = sd_df.apply(np.sum)

det_assets_df = all_assets_df.copy().drop("1_Sov", axis=1)
det_assets_df = pd.merge(sd_df.T, det_assets_df, left_index=True, right_index=True, how='inner')

asset_distr_df = det_assets_df.copy()
asset_distr_df.loc[:, "Bank_Country"] = asset_distr_df.index.get_values()
asset_distr_df.loc[:, "Bank_Country"] = asset_distr_df.loc[:, "Bank_Country"].apply(extract_country)
sign_countries_df = asset_distr_df.groupby("Bank_Country").sum()

sd_distr_df = np.round(sign_countries_df.T.div(sign_countries_df.apply(np.sum, axis=0), axis=0), 4)
sd_distr_df.columns = sd_distr_df.columns.rename('Bank Country')
sd_distr_df.index = sd_distr_df.index.rename('Asset')

,2_Fin,3_Corp,4_1_Ret_Res,4_2_Ret_Rev,4_3_Ret_SME,5_CRE
Bank,,,,,,
AT001,25044.000000,61237.000000,36663.000000,23153.000000,3467.000000,22228.000000
AT002,30360.807140,76431.863305,10137.189089,639.523821,1164.491564,3524.980441
AT003,10788.368246,13875.996362,1946.290036,2233.869131,4911.784578,1260.157675
BE004,228210.984799,87307.073348,24119.021273,1046.000000,5540.570508,1338.894636
BE005,23870.918756,59611.571457,60697.407380,436.367005,20538.963162,7169.788670
CY006,7907.000000,16160.000000,4590.000000,750.000000,983.000000,1697.000000
CY007,7293.692468,11923.110053,4728.000000,0.000000,1971.000000,4831.000000
DE017,194398.581091,391039.285129,121700.485615,652.607505,7972.369568,48298.594366
DE018,138190.100000,154597.500000,50916.000000,932.000000,6266.500000,71707.800000


In [46]:
pd.read_excel("Tier-1 CRs 2010.xlsx", header=2, index_col=1).ix["AT001", 2]

0.081000000000000003